In [3]:
import os
import pandas as pd
import empyrical as empy

import warnings
warnings.filterwarnings("ignore")

In [4]:
date_file = 'data/date_features.csv'
tradedates = pd.read_csv(date_file)['date']
tradedates = pd.to_datetime(tradedates).dt.date

t_assets = ['DBC', 'EEM', 'IYR', 'TLT', 'VTI', 'SPY', 'QQQ', 'VGK', 'EWJ', 'GLD', 'LQD', 'VYM', 'TIP']

In [5]:
def convert2trans_preds(real_preds, proba, tradedates):
    """
    preds: pdDataFrame() -> Simple Voting
    date DBC TLT
    2020-12-31 1 0

    proba: pdDataFrame() -> proba_1
    date DBC TLT
    2020-12-31 0.6 0.3
    """

    final_name = 'Trend'
    vote_name = 'Real Prediction'
    cl_name = 'confidence level'
    proba_1_name = 'proba_1'

    real_preds = real_preds.reindex(tradedates).dropna()
    print(real_preds)

    trans_preds = pd.DataFrame()
    for td in real_preds.index:
        # print(td)
        vote_df = real_preds.loc[td, :].T
        vote_df = vote_df.dropna()
        vote_df = vote_df.astype(int)
        vote_df.name = vote_name
        sub_proba = proba.loc[td, :].T
        sub_proba.name = proba_1_name
        vote_df = pd.concat([vote_df, sub_proba], axis=1)

        cl_ser = pd.Series(index=vote_df.index)
        for a in vote_df.index:
            trend = vote_df.loc[a, vote_name]
            proba_1 = vote_df.loc[a, proba_1_name]
            if trend == 0:
                cl_ser[a] = 1 - proba_1
            else:
                cl_ser[a] = proba_1
        cl_ser.name = cl_name
        vote_df = pd.concat([vote_df, cl_ser], axis=1)

        notin_asset = list(set(t_assets).difference(set(vote_df.index)))
        notin_df = pd.DataFrame(index=notin_asset, columns=vote_df.columns)
        vote_df = pd.concat([vote_df, notin_df])
        vote_df.index.name = 'Symbol'

        new_vote = vote_df.copy()

        td_idx = tradedates[tradedates == td].index[0]
        new_vote['Trading Date'] = tradedates[td_idx + 1]
        new_vote['Train Date'] = td
    #     print(new_vote)

        trans_preds = pd.concat([trans_preds, new_vote])
    return trans_preds

In [18]:
base_folder = './result/2022/ML-MPT_alignment/ML-preds/'
# Top 3 sharpe
# preds_cases = [
#     'Top5NNLow1_byAccuracy_MPT', 
#     'VoteAll_bySD',
#     'Top5NNLow1_byMSE_MPT'
# ]

# 
preds_cases = [
    'Top5NNLow1_byAccuracy_MPT',
    'Top5NNLow1_byMSE_MPT',
    # 'Top5VoteAll_byCV_MPT'
]

In [19]:
proba = {}
for c in preds_cases:
    file = os.path.join(base_folder, "{}.csv".format(c))
    df = pd.read_csv(file)
    df_pivot = pd.pivot_table(df, values='proba_1', index='Train Date', columns='Symbol')
    df_pivot.index = pd.to_datetime(df_pivot.index).date
    proba[c] = df_pivot
proba_df = pd.concat(proba.values(), keys=proba.keys(), axis=1).dropna()
proba_df

Top5NNLow1_byAccuracy_MPT                                         \
Symbol                           DBC       EEM    EWJ  GLD    IYR       LQD   
2007-07-31                       0.8  0.833333  0.875  1.0  0.875  0.285714   
2007-08-01                       0.8  1.000000  0.875  1.0  0.875  0.142857   
2007-08-02                       0.8  1.000000  0.875  1.0  0.875  0.142857   
2007-08-03                       0.8  1.000000  0.875  1.0  0.875  0.285714   
2007-08-06                       0.8  1.000000  0.875  1.0  0.875  0.142857   
...                              ...       ...    ...  ...    ...       ...   
2022-03-24                       0.0  0.750000  0.875  0.6  0.250  1.000000   
2022-03-25                       0.0  0.750000  0.875  0.6  0.250  1.000000   
2022-03-28                       0.0  0.750000  0.875  0.6  0.250  1.000000   
2022-03-29                       0.0  0.750000  0.875  0.6  0.250  1.000000   
2022-03-30                       0.0  0.750000  0.875  0.6  0.250  1.000000   

                                                    ... Top5NNLow1_byMSE_MPT  \
Symbol           QQQ       SPY       TIP       TLT  ...                  GLD   
2007-07-31  0.857143  1.000000  0.333333  0.285714  ...                  1.0   
2007-08-01  0.857143  1.000000  0.111111  0.285714  ...                  1.0   
2007-08-02  0.857143  0.857143  0.111111  0.285714  ...                  1.0   
2007-08-03  0.857143  1.000000  0.222222  0.285714  ...                  1.0   
2007-08-06  0.714286  1.000000  0.222222  0.285714  ...                  1.0   
...              ...       ...       ...       ...  ...                  ...   
2022-03-24  1.000000  0.714286  0.875000  0.166667  ...                  0.6   
2022-03-25  1.000000  0.714286  0.875000  0.166667  ...                  0.6   
2022-03-28  1.000000  0.714286  0.875000  0.166667  ...                  0.6   
2022-03-29  1.000000  0.714286  0.875000  0.166667  ...                  0.6   
2022-03-30  1.000000  0.714286  0.875000  0.166667  ...                  0.6   

                                                                            \
Symbol        IYR       LQD       QQQ       SPY    TIP       TLT       VGK   
2007-07-31  0.875  0.285714  0.857143  1.000000  0.375  0.285714  0.857143   
2007-08-01  0.875  0.142857  0.857143  1.000000  0.125  0.285714  0.857143   
2007-08-02  0.875  0.142857  0.857143  0.857143  0.125  0.285714  0.857143   
2007-08-03  0.875  0.285714  0.857143  1.000000  0.250  0.285714  0.857143   
2007-08-06  0.875  0.142857  0.857143  1.000000  0.250  0.285714  0.857143   
...           ...       ...       ...       ...    ...       ...       ...   
2022-03-24  0.250  1.000000  1.000000  0.833333  0.875  0.428571  0.625000   
2022-03-25  0.250  1.000000  1.000000  0.833333  0.875  0.428571  0.625000   
2022-03-28  0.250  1.000000  1.000000  0.833333  0.875  0.428571  0.625000   
2022-03-29  0.250  1.000000  1.000000  0.833333  0.875  0.428571  0.625000   
2022-03-30  0.250  1.000000  1.000000  0.833333  0.875  0.428571  0.625000   

                             
Symbol        VTI       VYM  
2007-07-31  0.625  0.000000  
2007-08-01  0.625  0.000000  
2007-08-02  0.625  0.000000  
2007-08-03  0.625  0.000000  
2007-08-06  0.625  0.000000  
...           ...       ...  
2022-03-24  1.000  0.222222  
2022-03-25  1.000  0.222222  
2022-03-28  1.000  0.222222  
2022-03-29  1.000  0.222222  
2022-03-30  1.000  0.222222  

[3694 rows x 26 columns]

In [20]:
vote_proba = pd.DataFrame()
for asset in t_assets:
    sub_proba_df = proba_df.loc[:, proba_df.columns.get_level_values(1)==asset]
    vote_proba[asset] = sub_proba_df.mean(axis=1)
vote_proba

,DBC,EEM,IYR,TLT,VTI,SPY,QQQ,VGK,EWJ,GLD,LQD,VYM,TIP
2007-07-31,0.8,0.833333,0.875,0.285714,0.625,1.000000,0.857143,0.857143,0.866071,1.0,0.285714,0.000000,0.354167
2007-08-01,0.8,1.000000,0.875,0.285714,0.625,1.000000,0.857143,0.857143,0.866071,1.0,0.142857,0.000000,0.118056
2007-08-02,0.8,1.000000,0.875,0.285714,0.625,0.857143,0.857143,0.857143,0.866071,1.0,0.142857,0.000000,0.118056
2007-08-03,0.8,1.000000,0.875,0.285714,0.625,1.000000,0.857143,0.857143,0.866071,1.0,0.285714,0.000000,0.236111
2007-08-06,0.8,1.000000,0.875,0.285714,0.625,1.000000,0.785714,0.857143,0.866071,1.0,0.142857,0.000000,0.236111
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,0.0,0.750000,0.250,0.297619,1.000,0.773810,1.000000,0.625000,0.875000,0.6,1.000000,0.222222,0.875000
2022-03-25,0.0,0.750000,0.250,0.297619,1.000,0.773810,1.000000,0.625000,0.875000,0.6,1.000000,0.222222,0.875000
2022-03-28,0.0,0.750000,0.250,0.297619,1.000,0.773810,1.000000,0.625000,0.875000,0.6,1.000000,0.222222,0.875000
2022-03-29,0.0,0.750000,0.250,0.297619,1.000,0.773810,1.000000,0.625000,0.875000,0.6,1.000000,0.222222,0.875000


In [21]:
vote_preds = (vote_proba>=0.5).astype(int)
vote_preds

,DBC,EEM,IYR,TLT,VTI,SPY,QQQ,VGK,EWJ,GLD,LQD,VYM,TIP
2007-07-31,1,1,1,0,1,1,1,1,1,1,0,0,0
2007-08-01,1,1,1,0,1,1,1,1,1,1,0,0,0
2007-08-02,1,1,1,0,1,1,1,1,1,1,0,0,0
2007-08-03,1,1,1,0,1,1,1,1,1,1,0,0,0
2007-08-06,1,1,1,0,1,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-24,0,1,0,0,1,1,1,1,1,1,1,0,1
2022-03-25,0,1,0,0,1,1,1,1,1,1,1,0,1
2022-03-28,0,1,0,0,1,1,1,1,1,1,1,0,1
2022-03-29,0,1,0,0,1,1,1,1,1,1,1,0,1


In [22]:
trans_preds = convert2trans_preds(vote_preds, vote_proba, tradedates)
fn = os.path.join(base_folder, 'trans_preds_voteEnsemble({}).csv'.format('+'.join(preds_cases)))
trans_preds.to_csv(fn)

            DBC  EEM  IYR  TLT  VTI  SPY  QQQ  VGK  EWJ  GLD  LQD  VYM  TIP
date                                                                       
2007-07-31  1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0
2007-08-01  1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0
2007-08-02  1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0
2007-08-03  1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0
2007-08-06  1.0  1.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0
...         ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
2022-03-24  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0
2022-03-25  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0
2022-03-28  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0
2022-03-29  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0
2022-03-30  0.0  1.0  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  0.0  1.0

[3694 rows 